# ML approach ( DL overfit like crazy)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount= True )

Mounted at /content/gdrive


## 1- Shape checks of the cleaned dataset


In [ ]:
import numpy as np
import os, random
from glob import glob

# Paths
train_dir = "/content/gdrive/MyDrive/modma_project/modma_per_subject/train"
test_dir = "/content/gdrive/MyDrive/modma_project/modma_per_subject/test"

# Pick a random file from train
sample_file = random.choice(glob(os.path.join(train_dir, "*.npz")))
print("📄 Random subject file:", sample_file)

# Load it
data = np.load(sample_file)
for k in data.files:
    print(f"{k}: shape = {data[k].shape}")

# Optional — quick data sanity checks
X = data["X"]
y = data["y"]
print("\nX dtype:", X.dtype, " | min:", X.min(), "max:", X.max())
print("y unique values:", np.unique(y))
# Re-executed to show shape of a random file as requested by the user.

📄 Random subject file: /content/gdrive/MyDrive/modma_project/modma_per_subject/train/subject_2010016.npz
X: shape = (480, 128, 251, 1)
y: shape = (480,)
subject: shape = ()

X dtype: float64  | min: -9.381604389357635 max: 21.95066005968129
y unique values: [1]


In [ ]:
from glob import glob
import numpy as np

train_dir = "/content/gdrive/MyDrive/modma_project/modma_per_subject/train"

for path in sorted(glob(os.path.join(train_dir, "*.npz"))[:5]):  # check first 5
    d = np.load(path)
    y = d["y"]
    print(os.path.basename(path), "unique labels:", np.unique(y))


subject_2010002.npz unique labels: [1]
subject_2010005.npz unique labels: [1]
subject_2010006.npz unique labels: [1]
subject_2010008.npz unique labels: [1]
subject_2010010.npz unique labels: [1]


In [ ]:
# Function to inspect a random subject file
import random

def inspect_random_subject(folder, n=3):
    files = sorted(glob(os.path.join(folder, "*.npz")))
    n = min(n, len(files))  # fix here
    sample_files = random.sample(files, n)
    for f in sample_files:
        data = np.load(f)
        print(f"\n📄 File: {os.path.basename(f)}")
        for k in data.files:
            print(f"  {k}: shape = {data[k].shape}, dtype = {data[k].dtype}")
        X = data["X"].squeeze(-1)
        y = data["y"]
        print(f"  -> After squeeze, X shape: {X.shape}")
        print(f"  -> Labels unique: {np.unique(y)}")


# Inspect 3 random train subjects
inspect_random_subject(TRAIN_DIR, n=3)

# Check class balance across all train subjects
def class_balance(folder):
    files = sorted(glob(os.path.join(folder, "*.npz")))
    if len(files) == 0:
        print("No files found in folder!")
        return
    all_labels = []
    for f in files:
        d = np.load(f)
        # pick the first label as representative of subject
        y = d["y"]
        if len(y) == 0:
            print(f"Warning: {os.path.basename(f)} has no labels!")
            continue
        all_labels.append(y[0])
    if len(all_labels) == 0:
        print("No labels found across subjects!")
        return
    all_labels = np.array(all_labels)
    unique, counts = np.unique(all_labels, return_counts=True)
    print("\nClass balance across subjects:")
    for u, c in zip(unique, counts):
        print(f"  Class {u}: {c} subjects")

# Example usage
TRAIN_DIR = "/content/gdrive/MyDrive/modma_project/modma_per_subject/train"
TEST_DIR  = "/content/gdrive/MyDrive/modma_project/modma_per_subject/test"
class_balance(TRAIN_DIR)
class_balance(TEST_DIR)



📄 File: subject_2010024.npz
  X: shape = (480, 128, 251, 1), dtype = float64
  y: shape = (480,), dtype = int64
  subject: shape = (), dtype = <U7
  -> After squeeze, X shape: (480, 128, 251)
  -> Labels unique: [1]

📄 File: subject_2030019.npz
  X: shape = (480, 128, 251, 1), dtype = float64
  y: shape = (480,), dtype = int64
  subject: shape = (), dtype = <U7
  -> After squeeze, X shape: (480, 128, 251)
  -> Labels unique: [0]

📄 File: subject_2020026.npz
  X: shape = (480, 128, 251, 1), dtype = float64
  y: shape = (480,), dtype = int64
  subject: shape = (), dtype = <U7
  -> After squeeze, X shape: (480, 128, 251)
  -> Labels unique: [0]

Class balance across subjects:
  Class 0: 25 subjects
  Class 1: 17 subjects

Class balance across subjects:
  Class 0: 4 subjects
  Class 1: 7 subjects


## 2-CSP + reimannian feature extraction + Global tangent space


In [ ]:
# Install dependencies (if not already)
!pip install mne scikit-learn numpy scipy pandas pyriemann xgboost joblib --quiet

# -----------------------------
# Import libraries
import numpy as np
import os
from glob import glob
from sklearn.model_selection import GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, classification_report, confusion_matrix
from mne.decoding import CSP
from mne.filter import filter_data

print("✅ Setup complete")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.7/127.7 kB 11.4 MB/s eta 0:00:00
✅ Setup complete


In [ ]:
# ---------------- CONFIG
TRAIN_DIR = "/content/gdrive/MyDrive/modma_project/modma_per_subject/train"
TEST_DIR  = "/content/gdrive/MyDrive/modma_project/modma_per_subject/test"
ART_DIR   = "/content/gdrive/MyDrive/modma_project/Ml_approach"

os.makedirs(ART_DIR, exist_ok=True)

CSP_COMPONENTS = 6
BAND = (8, 30)
SFREQ = 250
EPS = 1e-6
SEED = 42
np.random.seed(SEED)


In [ ]:
def load_subject_npz(path):
    d = np.load(path, allow_pickle=True)
    X = d["X"]
    if X.ndim == 4: X = X.squeeze(-1)   # (trials, ch, time)
    y = d["y"]
    subj = d["subject"]
    if getattr(subj, "shape", ()) == (): subj = subj.item()
    return X, y, subj

def list_npz(folder):
    return sorted(glob(os.path.join(folder, "*.npz")))

train_files = list_npz(TRAIN_DIR)
test_files  = list_npz(TEST_DIR)
print(f"Train subjects: {len(train_files)} | Test subjects: {len(test_files)}")


Train subjects: 42 | Test subjects: 11


In [ ]:
from mne.filter import filter_data

def bandpass_trials(X):
    Xf = np.zeros_like(X)
    for i in range(X.shape[0]):
        Xf[i] = filter_data(X[i], sfreq=SFREQ, l_freq=BAND[0], h_freq=BAND[1],
                            method='iir', verbose=False)
    return Xf

def fit_csp_on_train(train_files, n_components=6, max_trials_per_subj=20):
    X_fit, y_fit = [], []
    for f in train_files:
        X, y, _ = load_subject_npz(f)
        n = min(max_trials_per_subj, len(y))
        idx = np.random.choice(len(y), n, replace=False)
        Xf = bandpass_trials(X[idx])
        X_fit.append(Xf)
        y_fit.append(y[idx])
    X_fit = np.concatenate(X_fit, axis=0)
    y_fit = np.concatenate(y_fit, axis=0)
    csp = CSP(n_components=n_components, norm_trace=False, transform_into='csp_space')

    csp.fit(X_fit, y_fit)
    return csp

print("Fitting CSP on TRAIN only...")
csp = fit_csp_on_train(train_files, n_components=CSP_COMPONENTS)
joblib.dump(csp, os.path.join(ART_DIR, "csp_pipeline.pkl"))
print("✅ CSP saved.")


Fitting CSP on TRAIN only...
Computing rank from data with rank=None
    Using tolerance 19 (2.2e-16 eps * 128 dim * 6.8e+14  max singular value)
    Estimated rank (data): 128
    data: rank 128 computed from 128 data channels with 0 projectors
Reducing data rank from 128 -> 128
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
✅ CSP saved.


In [ ]:
def transform_csp(files, csp):
    X_list, y_list, g_list = [], [], []
    for f in files:
        X, y, subj = load_subject_npz(f)
        Xf = bandpass_trials(X)
        Xcsp = csp.transform(Xf)
        X_list.append(Xcsp); y_list.append(y); g_list.append(np.repeat(subj, len(y)))
    return np.concatenate(X_list), np.concatenate(y_list), np.concatenate(g_list)

print("Transforming CSP (train/test)...")
Xtr_csp, ytr, gtr = transform_csp(train_files, csp)
Xte_csp, yte, gte = transform_csp(test_files,  csp)
print("CSP shapes:", Xtr_csp.shape, Xte_csp.shape)


Transforming CSP (train/test)...
CSP shapes: (20160, 6, 251) (5280, 6, 251)


In [ ]:
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

def compute_global_ts_mean_cov(train_files):
    cov_est = Covariances(estimator='oas')
    covs_mean = None
    count = 0
    for f in train_files:
        X, y, _ = load_subject_npz(f)
        Xf = bandpass_trials(X)
        covs = cov_est.fit_transform(Xf)
        subj_cov = covs.mean(axis=0)
        covs_mean = subj_cov if covs_mean is None else (covs_mean + subj_cov)
        count += 1
    mean_cov = covs_mean / count
    return mean_cov

def fit_global_ts(mean_cov):
    ts = TangentSpace()
    ts.fit(np.expand_dims(mean_cov, axis=0))
    return ts

print("Computing global mean covariance...")
mean_cov = compute_global_ts_mean_cov(train_files)
ts = fit_global_ts(mean_cov)
joblib.dump(ts, os.path.join(ART_DIR, "global_tangent_space.pkl"))
print("✅ TangentSpace saved.")

Computing global mean covariance...
✅ TangentSpace saved.


In [ ]:
def transform_riemann(files, ts):
    cov_est = Covariances(estimator='oas')
    X_list, y_list, g_list = [], [], []
    for f in files:
        X, y, subj = load_subject_npz(f)
        Xf = bandpass_trials(X)
        covs = cov_est.fit_transform(Xf)
        covs += np.eye(covs.shape[1]) * EPS
        X_ts = ts.transform(covs)
        X_list.append(X_ts); y_list.append(y); g_list.append(np.repeat(subj, len(y)))
    return np.concatenate(X_list), np.concatenate(y_list), np.concatenate(g_list)

print("Transforming Riemann features...")
Xtr_riem, _, _ = transform_riemann(train_files, ts)
Xte_riem, _, _ = transform_riemann(test_files,  ts)
print("Riemann shapes:", Xtr_riem.shape, Xte_riem.shape)

Transforming Riemann features...
Riemann shapes: (20160, 8256) (5280, 8256)
Stats shapes: (20160, 4, 251) (5280, 4, 251)
CSP: (20160, 6, 251)
Riemann: (20160, 8256)
Stats: (20160, 4, 251)
Aligning to minimum length: 20160


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
# re-define fixed stats function
def stats_from_csp(X_csp):
    if X_csp.ndim == 3:
        mean = X_csp.mean(axis=2)
        std  = X_csp.std(axis=2)
        sk   = (((X_csp - mean[:, :, None]) / (std[:, :, None] + 1e-12))**3).mean(axis=2)
        kt   = (((X_csp - mean[:, :, None]) / (std[:, :, None] + 1e-12))**4).mean(axis=2)
        return np.hstack([mean, std, sk, kt])
    else:
        mean = X_csp.mean(axis=1, keepdims=True)
        std  = X_csp.std(axis=1, keepdims=True)
        sk   = (((X_csp - mean)/ (std + 1e-12))**3).mean(axis=1, keepdims=True)
        kt   = (((X_csp - mean)/ (std + 1e-12))**4).mean(axis=1, keepdims=True)
        return np.hstack([mean, std, sk, kt])

# recompute stats + fuse only
Xtr_stats = stats_from_csp(Xtr_csp)
Xte_stats = stats_from_csp(Xte_csp)
print("Stats shapes:", Xtr_stats.shape, Xte_stats.shape)
# Flatten CSP outputs if 3D (mean over time dimension)
if Xtr_csp.ndim == 3:
    Xtr_csp = Xtr_csp.mean(axis=2)
    Xte_csp = Xte_csp.mean(axis=2)

print("Flattened CSP shapes:", Xtr_csp.shape, Xte_csp.shape)


Xtr_final = np.hstack([Xtr_csp, Xtr_riem, Xtr_stats])
Xte_final = np.hstack([Xte_csp, Xte_riem, Xte_stats])
print("Fused:", Xtr_final.shape, Xte_final.shape)


Stats shapes: (20160, 24) (5280, 24)
Flattened CSP shapes: (20160, 6) (5280, 6)
Fused: (20160, 8286) (5280, 8286)


In [ ]:
#THIS CELL IS JUST A PATCH , to be run only when the ram crashes
import numpy as np

# Load pre-saved fused features
fused_path = "/content/gdrive/MyDrive/modma_project/Ml_approach/fused_features.npz"
data = np.load(fused_path)

Xtr_final = data["X_train"]
ytr = data["y_train"]
gtr = data["groups_train"]

Xte_final = data["X_test"]
yte = data["y_test"]
gte = data["groups_test"]

print("Loaded fused features:", Xtr_final.shape, Xte_final.shape)


Loaded fused features: (20160, 8286) (5280, 8286)


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

print("Fitting StandardScaler on TRAIN...")
scaler = StandardScaler()
scaler.fit(Xtr_final)
Xtr_scaled = scaler.transform(Xtr_final)
Xte_scaled = scaler.transform(Xte_final)
"""joblib.dump(scaler, os.path.join(ART_DIR, "scaler.pkl"))

np.savez(os.path.join(ART_DIR, "fused_features.npz"),
         X_train=Xtr_scaled, y_train=ytr, groups_train=gtr,
         X_test=Xte_scaled,  y_test=yte, groups_test=gte)
print("✅ Saved: scaler.pkl, fused_features.npz")"""

Fitting StandardScaler on TRAIN...


'joblib.dump(scaler, os.path.join(ART_DIR, "scaler.pkl"))\n\nnp.savez(os.path.join(ART_DIR, "fused_features.npz"),\n         X_train=Xtr_scaled, y_train=ytr, groups_train=gtr,\n         X_test=Xte_scaled,  y_test=yte, groups_test=gte)\nprint("✅ Saved: scaler.pkl, fused_features.npz")'

## 3-Prediction

In [ ]:
#XGB
import gc, xgboost as xgb, numpy as np
gc.collect()

print("RAM-safe XGBoost training...")

# light DMatrix creation (saves 30–40% memory)
dtrain = xgb.DMatrix(Xtr_final, label=ytr)

# use a stratified validation split
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
tr_idx, va_idx = next(sss.split(Xtr_final, ytr))
dtr = xgb.DMatrix(Xtr_final[tr_idx], label=ytr[tr_idx])
dva = xgb.DMatrix(Xtr_final[va_idx], label=ytr[va_idx])

params = dict(
    objective="binary:logistic",
    eval_metric="logloss",
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.2,
    reg_alpha=0.2,
    seed=42,
    tree_method="hist",   # 🧠 less RAM
    max_bin=256,
    nthread=2             # throttle CPU threads
)

bst = xgb.train(
    params,
    dtr,
    num_boost_round=500,
    evals=[(dtr, "train"), (dva, "val")],
    early_stopping_rounds=30,
    verbose_eval=25
)

model_path = "/content/gdrive/MyDrive/modma_project/Ml_approach/xgb_earlystop_model.json"
bst.save_model(model_path)
print("✅ Model saved to:", model_path)

# quick sanity check
dtest = xgb.DMatrix(Xte_final, label=yte)
preds = bst.predict(dtest)
y_pred = (preds >= 0.5).astype(int)
bal_acc = 0.5 * ((y_pred[yte==1]==1).mean() + (y_pred[yte==0]==0).mean())
print(f"Balanced Accuracy (test): {bal_acc:.4f}")


RAM-safe XGBoost training...
[0]	train-logloss:0.63034	val-logloss:0.63057
[25]	train-logloss:0.16703	val-logloss:0.16921
[50]	train-logloss:0.05364	val-logloss:0.05645
[75]	train-logloss:0.01901	val-logloss:0.02107
[100]	train-logloss:0.00743	val-logloss:0.00888
[125]	train-logloss:0.00328	val-logloss:0.00430
[150]	train-logloss:0.00173	val-logloss:0.00246
[175]	train-logloss:0.00105	val-logloss:0.00160
[200]	train-logloss:0.00073	val-logloss:0.00118
[225]	train-logloss:0.00056	val-logloss:0.00095
[250]	train-logloss:0.00045	val-logloss:0.00081
[275]	train-logloss:0.00038	val-logloss:0.00071
[300]	train-logloss:0.00034	val-logloss:0.00064
[325]	train-logloss:0.00030	val-logloss:0.00059
[350]	train-logloss:0.00028	val-logloss:0.00055
[375]	train-logloss:0.00026	val-logloss:0.00052
[400]	train-logloss:0.00024	val-logloss:0.00050
[425]	train-logloss:0.00023	val-logloss:0.00048
[450]	train-logloss:0.00022	val-logloss:0.00046
[475]	train-logloss:0.00021	val-logloss:0.00045
[499]	train-logl

In [ ]:
#SVM
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score

print("⚙️ Training SVM...")

svm_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, random_state=42))
])

svm_pipe.fit(Xtr_final, ytr)
preds = svm_pipe.predict(Xte_final)
bal_acc = balanced_accuracy_score(yte, preds)

print(f"✅ SVM Balanced Accuracy (test): {bal_acc:.4f}")


⚙️ Training SVM...
✅ SVM Balanced Accuracy (test): 0.6454


In [ ]:
import joblib
joblib.dump(svm_pipe, "/content/gdrive/MyDrive/modma_project/Ml_approach/svm_model.pkl")


['/content/gdrive/MyDrive/modma_project/Ml_approach/svm_model.pkl']